In [65]:
import yfinance as yf

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [66]:
df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-01-01")
df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-01-01")
df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-01-01")


apple_path = 'datasets/apple.csv'
microsoft_path = 'datasets/microsoft.csv'
amazon_path = 'datasets/amazon.csv'

df_apple_to_csv.to_csv(apple_path)
df_microsoft_to_csv.to_csv(microsoft_path)
df_amazon_to_csv.to_csv(amazon_path)

df_apple = pd.read_csv(apple_path)
df_microsoft = pd.read_csv(microsoft_path)
df_amazon = pd.read_csv(amazon_path)

C:\Users\PLPASIG\AppData\Local\Temp\ipykernel_2168\4222972167.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed
C:\Users\PLPASIG\AppData\Local\Temp\ipykernel_2168\4222972167.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed
C:\Users\PLPASIG\AppData\Local\Temp\ipykernel_2168\4222972167.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed


In [67]:
df_apple.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,72.53850555419922,72.59888386623527,71.29229630932706,71.54588227171874,135480400
3,2020-01-03,71.83329772949219,72.59406313642577,71.60869213351766,71.76567430155144,146322800
4,2020-01-06,72.40567779541016,72.44432080433776,70.7030121336898,70.95418800651902,118387200


In [68]:
df_microsoft.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,153.0422821044922,153.1470931177711,150.86032587757265,151.28909291774593,22622100
3,2020-01-03,151.13662719726562,152.40388225400224,150.60304918156822,150.85079221664952,21116200
4,2020-01-06,151.52732849121094,151.59403303057528,149.1262122356397,149.66932792275708,20813700


In [69]:
df_amazon.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AMZN,AMZN,AMZN,AMZN,AMZN
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000
3,2020-01-03,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000
4,2020-01-06,95.14399719238281,95.18450164794922,93.0,93.0,81236000


In [70]:
df_apple['Date'] = df_apple['Price']
df_apple = df_apple.drop(index=[0,1], columns='Price')

df_microsoft['Date'] = df_microsoft['Price']
df_microsoft = df_microsoft.drop(index=[0,1], columns='Price')

df_amazon['Date'] = df_amazon['Price']
df_amazon = df_amazon.drop(index=[0,1], columns='Price')

In [71]:
df_apple.head()

,Close,High,Low,Open,Volume,Date
2,72.53850555419922,72.59888386623527,71.29229630932706,71.54588227171874,135480400,2020-01-02
3,71.83329772949219,72.59406313642577,71.60869213351766,71.76567430155144,146322800,2020-01-03
4,72.40567779541016,72.44432080433776,70.7030121336898,70.95418800651902,118387200,2020-01-06
5,72.06513214111328,72.67132530208438,71.84535422985819,72.41532188246822,108872000,2020-01-07
6,73.22439575195312,73.5262872851597,71.76807129058055,71.76807129058055,132079200,2020-01-08


In [72]:
df_microsoft.head()

,Close,High,Low,Open,Volume,Date
2,153.0422821044922,153.1470931177711,150.86032587757265,151.28909291774593,22622100,2020-01-02
3,151.13662719726562,152.40388225400224,150.60304918156822,150.85079221664952,21116200,2020-01-03
4,151.52732849121094,151.59403303057528,149.1262122356397,149.66932792275708,20813700,2020-01-06
5,150.14573669433594,152.1371317710887,149.89800813543405,151.8036526962878,21634100,2020-01-07
6,152.5373077392578,153.21381792157746,150.49826874289445,151.43203046001807,27746500,2020-01-08


In [73]:

df_amazon.head()

,Close,High,Low,Open,Volume,Date
2,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000,2020-01-02
3,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000,2020-01-03
4,95.14399719238281,95.18450164794922,93.0,93.0,81236000,2020-01-06
5,95.34300231933594,95.69450378417969,94.60199737548828,95.2249984741211,80898000,2020-01-07
6,94.59850311279297,95.55000305175781,94.3219985961914,94.9020004272461,70160000,2020-01-08
